# Simplifying multilinear polynomial stings

https://www.codewars.com/kata/55f89832ac9a66518f000118/train/python

When we attended middle school were asked to simplify mathematical expressions like "3x-yx+2xy-x" (or usually bigger), and that was easy-peasy ("2x+xy"). But tell that to your pc and we'll see!

Write a function: simplify, that takes a string in input, representing a multilinear non-constant polynomial in integers coefficients (like "3x-zx+2xy-x"), and returns another string as output where the same expression has been simplified in the following way ( -> means application of simplify):

All possible sums and subtraction of equivalent monomials ("xy==yx") has been done, e.g.:
"cb+cba" -> "bc+abc", "2xy-yx" -> "xy", "-a+5ab+3a-c-2a" -> "-c+5ab"


All monomials appears in order of increasing number of variables, e.g.:
"-abc+3a+2ac" -> "3a+2ac-abc", "xyz-xz" -> "-xz+xyz"

If two monomials have the same number of variables, they appears in lexicographic order, e.g.:
"a+ca-ab" -> "a-ab+ac", "xzy+zby" ->"byz+xyz"

There is no leading + sign if the first coefficient is positive, e.g.:
"-y+x" -> "x-y", but no restrictions for -: "y-x" ->"-x+y"

N.B. to keep it simplest, the string in input is restricted to represent only multilinear non-constant polynomials, so you won't find something like `-3+yx^2'. Multilinear means in this context: of degree 1 on each variable.

Warning: the string in input can contain arbitrary variables represented by lowercase characters in the english alphabet.

## Start with parsing the input string

We need a terms array from the input string, tokenizing into elements
"cb+cba" -> ["bc", "+abc"], ["2xy-yx" -> ["2xy","yx"], "-a+5ab+3a-c-2a" -> ["-a","+5ab", "+3a", "-c", "-2a"]

In [408]:
import re

TOKEN_REGEX = re.compile(
    r'''
    (?P<sign>[+-]?)       # sign group: 0 or 1 + or - characters
    (?P<constant>\d*)     # constant group: 0 or more digits
    (?P<algebraic>[a-z]+) # algebraic group: 1 or more lowercase letters
    ''',
    re.X
)

for match in TOKEN_REGEX.finditer("+d-54abc+4rqs"):
    print(
        match.group("sign"),
        match.group("constant"),
        match.group("algebraic")
    )


+  d
- 54 abc
+ 4 rqs


## Now let's build a class that will describe a single multilinear term

We want to take the tokenized input and turn it into something that we can work with.

A term is uniquely defined by its constant and its algebraic part

```python
term = Term(2, 'a')
term.constant # should be 2
term.algebraic # should be 'a'

term = Term(-5, 'bc')
term.constant # should be -5
term.algebraic # should be 'bc'

```

In [407]:
class Term():
    def __init__(self, constant=0, algebraic=''):
        self.constant = constant
        self.algebraic = algebraic
   
    def __str__(self):
        return f"{self.constant}{self.algebraic}"

str(Term(10, 'asxd'))


'10asxd'

## Creating terms from the parsed tokens

Our regular expression produces 3 tokens, we need a method to create a method to produce a Term instance from the output of the regular expression

In [412]:
class Term():
    def __init__(self, constant=0, algebraic=''):
        self.constant = constant
        self.algebraic = algebraic

    @staticmethod
    def from_match(match):
        if match.group('sign') not in "+-":
            raise ValueError("sign must be + or -")
        constant = 1 if not match.group('constant') else int(match.group('constant'))
        sign_multiplier = -1 if match.group('sign') == '-' else 1
        return Term(
            constant * sign_multiplier,
            match.group('algebraic')
        )

    def __str__(self):
        return f"{self.constant}{self.algebraic}"

print(Term.from_match(TOKEN_REGEX.match("-4ax  ")))
print(Term.from_match(TOKEN_REGEX.match("-3asd")))
print(Term.from_match(TOKEN_REGEX.match("xyz")))


-4ax
-3asd
1xyz


In [413]:
for match in TOKEN_REGEX.finditer("+d-54abc+4rqs"):
    print(Term.from_match(match))

1d
-54abc
4rqs


## Sums and subtraction of equivalent monomials

'xy' is the same thing as 'yx', and 'zxa' is the same as 'axz', they are equivalent monomials

We should just be able to sort the letters in the multilinear expression (strings sort lexicographically). Terms are monomially equivalent if their multiliner expressions match

Summing terms should fail if not monomially equivalent, and just summ the constants if they are

In [430]:
class Term():
    def __init__(self, constant=0, algebraic=''):
        self.constant = constant
        self.algebraic = algebraic

    @staticmethod
    def from_match(match):
        if not match.group('constant'):
            constant = 1
        else:
            constant = int(match.group('constant'))
            
        if match.group('sign') == '-':
            sign_multiplier = -1
        else:
            sign_multiplier = 1

        return Term(
            constant * sign_multiplier,
            match.group('algebraic')
        )

    def __str__(self):
        return f"{self.constant}{self.algebraic}"

    def monomially_equivalent(self, other_term):
        return sorted(self.algebraic) == sorted(other_term.algebraic)

    def __add__(self, other):
        if not self.monomially_equivalent(other):
            raise TypeError()
        new_constant = self.constant + other.constant
        return Term(new_constant, self.algebraic)

#print(Term(4, 'adz').monomially_equivalent(Term(2, 'zad')))
print(Term(3, 'ab') + Term(4, 'ba'))


7ab


## All possible sums and subtraction of...

We need to take a collection of terms, and combine the monomially equivalent terms (by summing/subtracting their constants

So we'll introduce another class to deal with a term collection which we can inherit from list to give us access to useful list methods

In [438]:
class Expression(list):
    @staticmethod
    def from_poly(poly):
        exp = Expression()
        for match in TOKEN_REGEX.finditer(poly):
            exp.append(Term.from_match(match))
        return exp
    
    def simplified(self):
        # returns a new expression which is simplified according to our rules
        simplified = Expression()
        for existing_term in self:
            for simplified_term in simplified:
                try:
                    new_term = existing_term + simplified_term
                    simplified.remove(simplified_term)
                    simplified.append(new_term)
                    break
                except TypeError:
                    pass
            else:
                simplified.append(existing_term)  
        return simplified

exp = Expression()
exp.append(Term(4, "xz"))
exp.append(Term(3, "ab"))
exp.append(Term(4, "ba"))
exp.append(Term(-1, "zx"))
for t in exp.simplified(): print(t)
#exp.simplified()



#for t in exp2: print(t)
#exp2.simplified()

7ba
3zx


In [436]:
exp2 = Expression.from_poly("2xy-yx")
for t in exp2.simplified(): print(t)


1yx


## Ordering terms
All monomials appears in order of increasing number of variables
If two monomials have the same number of variables, they appears in lexicographic order.

We can implement a sort of terms by adding methods to the original Term class

In [439]:
class Term():
    def __init__(self, constant=0, algebraic=''):
        self.constant = constant
        self.algebraic = algebraic

    @staticmethod
    def from_match(match):
        if match.group('sign') not in "+-":
            raise ValueError("sign must be + or -")
        constant = 1 if not match.group('constant') else int(match.group('constant'))
        sign_multiplier = (-1 if match.group('sign') == '-' else 1)
        constant = constant if constant else 1
        return Term(
            constant * sign_multiplier,
            match.group('algebraic')
        )

    def __str__(self):
        return f"{self.constant}{self.algebraic}"

    def algebraic_normalized(self):
        return "".join(sorted(self.algebraic))
                       
    def monomially_equivalent(self, other_term):
        return self.algebraic_normalized() == other_term.algebraic_normalized()

    def __add__(self, other):
        if not self.monomially_equivalent(other):
            raise TypeError()
        new_constant = self.constant + other.constant
        return Term(new_constant, self.algebraic)

    def __gt__(self, other):
        if len(self.algebraic) > len(other.algebraic):
            return True
        elif len(self.algebraic) < len(other.algebraic):
            return False
        else:
            return self.algebraic_normalized() > other.algebraic_normalized()
        
tst = [Term(1,'ab'), Term(1,'xb'),Term(-1,'xy'),Term(-1,'c'), Term(2,'xyz'), Term(2,'xaz'),Term(1, 'a')]
for t in sorted(tst): print(t)

1a
-1c
1ab
1xb
-1xy
2xaz
2xyz


## We're really close now, just the final output to go

In [443]:
class Term():
    def __init__(self, constant=0, algebraic=''):
        self.constant = constant
        self.algebraic = algebraic

    @staticmethod
    def from_match(match):
        if match.group('sign') not in "+-":
            raise ValueError("sign must be + or -")
        constant = 1 if not match.group('constant') else int(match.group('constant'))
        sign_multiplier = (-1 if match.group('sign') == '-' else 1)
        constant = constant if constant else 1
        return Term(
            constant * sign_multiplier,
            match.group('algebraic')
        )

    def __str__(self):
        return f"{self.constant}{self.algebraic}"

    def algebraic_normalized(self):
        return "".join(sorted(self.algebraic))
                       
    def monomially_equivalent(self, other_term):
        return self.algebraic_normalized() == other_term.algebraic_normalized()

    def __add__(self, other):
        if not self.monomially_equivalent(other):
            raise TypeError()
        new_constant = self.constant + other.constant
        return Term(new_constant, self.algebraic)

    def __gt__(self, other):
        if len(self.algebraic) > len(other.algebraic):
            return True
        elif len(self.algebraic) < len(other.algebraic):
            return False
        else:
            return self.algebraic_normalized() > other.algebraic_normalized()

    def pretty_print(self):
        if self.constant == 0:
            return ""
        constant = "" if self.constant in [1, -1] else self.constant
        #print(constant)
        sign = "+" if self.constant > 0 else "-"
        return f"{sign}{constant}{self.algebraic_normalized()}"
        
class Expression(list):
    @staticmethod
    def from_poly(poly):
        exp = Expression()
        for match in TOKEN_REGEX.finditer(poly):
            exp.append(Term.from_match(match))
        return exp
    
    def simplified(self):
        simplified = Expression()
        for existing_term in self:
            for simplified_term in simplified:
                try:
                    new_term = existing_term + simplified_term
                    simplified.remove(simplified_term)
                    simplified.append(new_term)
                    break
                except TypeError:
                    pass
            else:
                simplified.append(existing_term)  
        return simplified

    def __str__(self):
        s = sorted(self.simplified())
        pretty = "".join([
            t.pretty_print()
            for t in s
        ])
        return pretty[1:] if pretty[0] == "+" else pretty

e = Expression.from_poly("3a+2a+2ac-abc+3abc")
str(e)

'5a+2ac+2abc'

In [444]:
def test(s):
    print(f'"{s}" -> "{Expression.from_poly(s)}"')
print('"3x-yx+2xy-x" -> "2x+xy"')
test("3x-yx+2xy-x")
print('"cb+cba" -> "bc+abc"')
test("cb+cba")
print('"2xy-yx" -> "xy"')
test("2xy-yx")
print('"-a+5ab+3a-c-2a" -> "-c+5ab"')
test("-a+5ab+3a-c-2a")
print('"a+ca-ab" -> "a-ab+ac"')
test("a+ca-ab")
print('"-abc+3a+2ac" -> "3a+2ac-abc"')
test("-abc+3a+2ac")
print('"xyz-xz" -> "-xz+xyz"')
test("xyz-xz")
print('"xzy+zby" ->"byz+xyz"')
test("xzy+zby")
print('"-y+x" -> "x-y"')
test("-y+x")
print('"y-x" ->"-x+y"')
test("y-x")

"3x-yx+2xy-x" -> "2x+xy"
"3x-yx+2xy-x" -> "2x+xy"
"cb+cba" -> "bc+abc"
"cb+cba" -> "bc+abc"
"2xy-yx" -> "xy"
"2xy-yx" -> "xy"
"-a+5ab+3a-c-2a" -> "-c+5ab"
"-a+5ab+3a-c-2a" -> "-c+5ab"
"a+ca-ab" -> "a-ab+ac"
"a+ca-ab" -> "a-ab+ac"
"-abc+3a+2ac" -> "3a+2ac-abc"
"-abc+3a+2ac" -> "3a+2ac-abc"
"xyz-xz" -> "-xz+xyz"
"xyz-xz" -> "-xz+xyz"
"xzy+zby" ->"byz+xyz"
"xzy+zby" -> "byz+xyz"
"-y+x" -> "x-y"
"-y+x" -> "x-y"
"y-x" ->"-x+y"
"y-x" -> "-x+y"
